In [1]:
# Install necessary libraries
!pip install pandas scikit-learn gradio


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import essential libraries
import pandas as pd
import smtplib
import gradio as gr
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import IsolationForest


In [4]:
# Load dataset (Ensure your file is uploaded in the same directory)
df = pd.read_csv("/synthetic_transaction_data.csv")

# Convert 'Date' column to datetime
df["Date"] = pd.to_datetime(df["Date"])

# Encode 'Location' column using LabelEncoder
le = LabelEncoder()
df["Location"] = le.fit_transform(df["Location"])

# Standardize numerical features using StandardScaler
scaler = StandardScaler()
df[["Transaction_Amount", "Location"]] = scaler.fit_transform(df[["Transaction_Amount", "Location"]])


In [5]:
# Train Isolation Forest for fraud detection
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(df[["Transaction_Amount", "Location"]])


IsolationForest(contamination=0.05, random_state=42)

In [6]:
# Email credentials and server details
EMAIL_ADDRESS = "ybshemashree.csbs2024@citchennai.net"          # Sender's email
EMAIL_PASSWORD = "gphr tzax bhos qpjv"               # App password (Not your regular password)
TO_EMAIL = "WizzKidx76@gmail.com"    # Recipient's email
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587


In [7]:
# Function to send email when fraud is detected
def send_email(subject, message):
    try:
        # Create email message
        msg = MIMEMultipart()
        msg["From"] = EMAIL_ADDRESS
        msg["To"] = TO_EMAIL
        msg["Subject"] = subject
        msg.attach(MIMEText(message, "plain"))

        # Connect to SMTP server and send email
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT, timeout=10) as server:
            server.starttls()  # Secure connection
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.sendmail(EMAIL_ADDRESS, TO_EMAIL, msg.as_string())

        print("✅ Email sent successfully!")

    except Exception as e:
        print(f"❌ Error: {e}")


In [8]:
# Function to detect fraud and send email if fraud is detected
def detect_fraud(date, transaction_amount, location):
    try:
        # Convert input date
        date = pd.to_datetime(date)

        # Encode location using the same encoder
        location_encoded = le.transform([location])[0]

        # Standardize input features
        input_data = scaler.transform([[float(transaction_amount), location_encoded]])

        # Predict using Isolation Forest
        prediction = model.predict(input_data)[0]
        result = "✅ Transaction Approved" if prediction == 1 else "🚨 Fraud Detected!"

        # Send email if fraud is detected
        if prediction == -1:
            send_email(
                subject="🚨 Fraud Alert: Suspicious Transaction Detected!",
                message=f"⚠️ Suspicious transaction detected on your account!\n\n"
                        f"📅 Date: {date.date()}\n"
                        f"💸 Amount: {transaction_amount}\n"
                        f"📍 Location: {location}\n\n"
                        f"Please review your account immediately!"
            )

        return result

    except Exception as e:
        return f"❌ Error: {e}"


In [9]:
# Build Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 💳 Fraud Detection System")

    # Input Fields
    date_input = gr.Textbox(label="📅 Date (YYYY-MM-DD)")
    amount_input = gr.Number(label="💸 Transaction Amount")
    location_input = gr.Textbox(label="📍 Location")

    # Button and Output
    submit_button = gr.Button("🔍 Detect Fraud")
    output_text = gr.Textbox(label="🧾 Result")

    # On button click, call detect_fraud() and show result
    submit_button.click(detect_fraud, inputs=[date_input, amount_input, location_input], outputs=output_text)


In [10]:
app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b879375071f360fd23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
